In [1]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
from app.utils.loader import load_docs

In [3]:
data_chunks = load_docs("app/data/prince_rag_knowledge_base.txt")

In [4]:
print(data_chunks[0].page_content)

ABOUT ME – PRINCE KUMAR

Prince Kumar is a Full Stack Python Developer and AI/ML Engineer currently pursuing a B.Tech in Mechanical Engineering at NIT Jalandhar (2023–2027).
He has strong interest in Artificial Intelligence, Machine Learning, Deep Learning, Computer Vision, NLP, and Full Stack Development.


In [5]:
import os
from pinecone import Pinecone, ServerlessSpec


api_key = os.getenv("PINECONE_API_KEY")
print("api_key:", api_key)

pc = Pinecone(api_key=api_key)

api_key: pcsk_e5cfc_1HBPr3Wuk5W6AfJuqgz1BiTBvjA9biAGmRU5rpkCLesX6LiyomGNNo5QgfyGZw


In [7]:
EMBEDDING_DIM = 384

In [8]:
# Load index name
index_name = os.getenv("PINECONE_INDEX_NAME")
print("pinecone_index_name:", index_name)

# Delete existing index if it exists to recreate with correct dimension
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print("\033[93mExisting index deleted.\033[0m")

pc.create_index(
    name=index_name,
    dimension=EMBEDDING_DIM,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region=os.getenv("PINECONE_ENVIRONMENT")
    )
)
print("\033[93mIndex created with dimension", EMBEDDING_DIM, "\033[0m")

pinecone_index_name: portdolio-info
Index created with dimension 384 


In [9]:
index = pc.Index(index_name)

In [56]:
# from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001"
# )

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

# Load a free sentence-transformers model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
# Get embedding dimension
test_embedding = embeddings.embed_query("test")
EMBEDDING_DIM = len(test_embedding)
print(f"Embedding dimension: {EMBEDDING_DIM}")

Embedding dimension: 384


In [ ]:
test_embedding

In [13]:
from langchain_pinecone import PineconeVectorStore

index_name = os.getenv("PINECONE_INDEX_NAME")
# Create vector store
vectorstore = PineconeVectorStore.from_documents(
    documents=data_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [15]:
# Example search in the vector store
query = "give me linkin id"
results = vectorstore.similarity_search(query, k=3)  # k=3 means top 3 results

print("Search results for query:", query)
for i, doc in enumerate(results):
    print(f"\nResult {i+1}:")
    print(doc.page_content)

Search results for query: give me linkin id

Result 1:
LEADERSHIP

- Core Member, Deja Vu Club, NIT Jalandhar (2025–2026)

ONLINE PRESENCE

GitHub: https://github.com/princekumar983777
LinkedIn: https://www.linkedin.com/in/prince-kumar-021460290

Result 2:
Deep Learning and Computer Vision
- Develop CNN-based models
- Implement face recognition and emotion detection systems
- Build real-time computer vision applications using OpenCV

NLP and RAG Systems
- Build RAG-based chatbots
- Implement semantic search and vector embeddings
- Use Hugging Face Transformers for NLP tasks
- Design domain-specific AI assistants

Result 3:
INTERNSHIP

AI/ML Intern – AD Infocom Systems, Nagpur (Remote)
June 2025 – July 2025
- Worked with real-world datasets
- Implemented ML algorithms
- Built and tested ML models using Python

ACHIEVEMENTS

- 1st rank in college-level hackathon
- Built 10+ ML mini-projects
- Diamond Rank on Google Skill Boost
- Solved 200+ DSA problems
- Participated in multiple hackath

In [ ]:
# Function to perform semantic search and generate response with Gemini
def search_and_respond(query):
    # Step 1: Search the vector store for relevant documents
    search_results = vectorstore.similarity_search(query, k=3)  # Get top 3 relevant chunks

    # Step 2: Extract the content from search results
    context = "\n\n".join([doc.page_content for doc in search_results])

    # Step 3: Create a prompt that includes the context
    prompt = f"""Use the following context to answer the question. If the context doesn't contain enough information, say so.

Context:
{context}

Question: {query}

Answer:"""

    # Step 4: Use Gemini to generate the response
    client = genai.Client()
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        generation_config={
            "temperature": 0.2,
            "top_p": 0.9,
            "max_output_tokens": 512,
        }
    )

    return response.text, search_results

# Example usage
query = "What are Prince Kumar's skills?"
response, sources = search_and_respond(query)

print("Query:", query)
print("\nResponse:")
print(response)
print("\nSources used:")
for i, doc in enumerate(sources):
    print(f"{i+1}. {doc.page_content[:100]}...")

In [ ]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="How does AI work?"
)
print(response.text)

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="How does AI work?"
)
print(response.text)

In [6]:

client = genai.Client()
chat = client.chats.create(model="gemini-2.5-flash")

response = chat.send_message("I have 2 dogs in my house.")
print(response.text)

response = chat.send_message("How many paws are in my house?")
print(response.text)

for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Oh, how lovely! What kind of dogs are they?
Assuming both your dogs have all their paws, that would be:

2 dogs * 4 paws/dog = **8 paws** in your house!
role - user: I have 2 dogs in my house.
role - model: Oh, how lovely! What kind of dogs are they?
role - user: How many paws are in my house?
role - model: Assuming both your dogs have all their paws, that would be:

2 dogs * 4 paws/dog = **8 paws** in your house!


In [11]:
file = open("app/data/prince_rag_knowledge_base.txt", "r", encoding="utf-8")
knowledge_base = file.read()
print(knowledge_base)

ABOUT ME – PRINCE KUMAR

Prince Kumar is a Full Stack Python Developer and AI/ML Engineer currently pursuing a B.Tech in Mechanical Engineering at NIT Jalandhar (2023–2027).
He has strong interest in Artificial Intelligence, Machine Learning, Deep Learning, Computer Vision, NLP, and Full Stack Development.

He focuses on building real-world AI-powered applications by combining backend systems, APIs, databases, and modern AI models.
He emphasizes practical implementation, clean architecture, and scalable solutions.

PROFESSIONAL OBJECTIVE

Prince aims to work on AI-driven software systems that solve real-world problems using Python, Machine Learning, Deep Learning, and Full Stack technologies.
He is actively seeking internships, freelance projects, and collaborative opportunities.

CORE EXPERTISE

- Full Stack Python Development
- Machine Learning and Data Science
- Deep Learning and Computer Vision
- NLP and Retrieval-Augmented Generation (RAG)
- Backend APIs and System Design
- Databa

In [12]:

client = genai.Client()

chat = client.chats.create(
    model="gemini-2.5-flash",
    system_instruction="You are Prince Kumar. A backend developer and AI enthusiast. Use the below knowledge base to answer the questions.\n\n" + knowledge_base,
    generation_config={
        "temperature": 0.2,
        "top_p": 0.9,
        "max_output_tokens": 256,
    }
)

response = chat.send_message("tell me about yourself.")
print(response.text)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


TypeError: Chats.create() got an unexpected keyword argument 'system_instruction'

In [49]:
!pip install --upgrade pinecone

  Using cached pinecone-8.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached pinecone_plugin_assistant-3.0.1-py3-none-any.whl.metadata (30 kB)
Using cached pinecone-8.0.0-py3-none-any.whl (745 kB)
Using cached pinecone_plugin_assistant-3.0.1-py3-none-any.whl (280 kB)

  Attempting uninstall: pinecone-plugin-assistant

    Found existing installation: pinecone-plugin-assistant 1.8.0

    Uninstalling pinecone-plugin-assistant-1.8.0:

      Successfully uninstalled pinecone-plugin-assistant-1.8.0

   ---------------------------------------- 0/2 [pinecone-plugin-assistant]
   ---------------------------------------- 0/2 [pinecone-plugin-assistant]
   ---------------------------------------- 0/2 [pinecone-plugin-assistant]
   ---------------------------------------- 0/2 [pinecone-plugin-assistant]
   ---------------------------------------- 0/2 [pinecone-plugin-assistant]
   ---------------------------------------- 0/2 [pinecone-plugin-assistant]
   --------------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.2.13 requires pinecone[asyncio]<8.0.0,>=6.0.0, but you have pinecone 8.0.0 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
